# KRAS Mutation Trans Effect on Phosphoproteomics

This notebook analyzes the trans effect of KRAS mutation on interacting and other proteins Phosphoproteomics, in Endometrial and Colon cancer.

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import collections
import re
import gseapy as gp
from gseapy.plot import barplot, dotplot

import cptac
import cptac.algorithms as al

en = cptac.Endometrial()
co = cptac.Colon()

Checking that index is up-to-date...Loading acetylproteomics data...Loading clinical data...Loading CNA data...Loading definitions data...Loading miRNA data...Loading phosphoproteomics_gene data...Loading phosphoproteomics_site data...Loading proteomics data...Loading somatic_binary data...Loading somatic data...Loading transcriptomics_circular data...Loading transcriptomics_linear data...Formatting dataframes...Checking that index is up-to-date...Loading clinical data...Loading miRNA data...Loading mutation_binary data...Loading mutation data...Loading phosphoproteomics_normal data...Loading phosphoproteomics_tumor data...Loading proteomics_normal data...Loading proteomics_tumor data...Loading transcriptomics data...Formatting dataframes...

### Investigate KRAS gene

In [2]:
gene = "KRAS"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
#omics = "proteomics"
#omics = "transcriptomics"
omics = "phosphoproteomics"
#omics = "acetylproteomics"

# Interacting Proteins: Phosphoproteomics

## Endometrial

### Generate interacting protein list

In [8]:
'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

omics_object = en.get_phosphoproteomics()

print("Generating interacting protein list")
interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in interacting_proteins:
    if omics == 'phosphoproteomics' or omics == 'acetylproteomics':
        col_regex = ip + "-.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics_object.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)

print("Interacting Proteins:")
for interacting_protein in interacting_proteins_in_omics_df:
    print(interacting_protein)

Generating interacting protein list
Interacting Proteins:
RAF1
MAPK1
KRAS
MAP2K2
MAPK3
PIK3CA
ERBB3
ERBB2
EGFR
BRAF
ARAF
MAP2K1
PTPN11
NF1
PIK3CG
NRAS
RALGDS
SRC
GRB2
SOS1
RAP1GDS1
HNRNPC
RASSF2
RASSF5


### Test for significant comparisons in any of interacting proteins

In [9]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = en.append_mutations_to_omics(mutation_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins_in_omics_df)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)


'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")

Doing t-test comparisons



/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3157: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


                    Comparison   P_Value
0  RAF1-T330_phosphoproteomics  0.000008
1  ARAF-S260_phosphoproteomics  0.000021





## Colon

### Generate interacting protein list

In [18]:
'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

omics_object = co.get_phosphoproteomics()

print("Generating interacting protein list")
interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in interacting_proteins:
    if omics == 'phosphoproteomics' or omics == 'acetylproteomics':
        col_regex = ip + "_.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics_object.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)

print("Interacting Proteins:")
for interacting_protein in interacting_proteins_in_omics_df:
    print(interacting_protein)

Generating interacting protein list
Interacting Proteins:
RAF1
MAPK1
MAP2K2
MAPK3
ERBB3
ERBB2
EGFR
BRAF
ARAF
MAP2K1
PTPN11
RET
NF1
SRC
SOS1
PRRT2
HNRNPC
RASSF2


### Test for significant comparisons in any of interacting proteins

In [19]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = co.append_mutations_to_omics(mutation_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins_in_omics_df)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)

'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")

Doing t-test comparisons



/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3157: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


No significant comparisons.


# All Proteins: Phosphoproteomics

## Endometrial

In [25]:
try:
    print("\nGene: ", gene)

    '''Use all proteins'''
    proteomics = en.get_proteomics()
    all_proteins = proteomics.columns
    all_proteins = list(all_proteins)

    all_proteins_no_dash = []
    for ap in all_proteins:
        if '-' not in ap:
            all_proteins_no_dash.append(ap)
            
    #all_proteins = all_proteins[:500]

    '''Create dataframe in order to do comparisons with wrap_ttest'''
    protdf = en.append_mutations_to_omics(mutation_genes=[gene], omics_df_name=omics, omics_genes=all_proteins_no_dash)
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    '''Create the binary valued column needed to do the comparison'''
    for ind, row in protdf.iterrows():
        if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
            protdf.at[ind,'Label'] = 'Mutated'
        else:
            protdf.at[ind,'Label'] = 'Wildtype'

    '''Format the dataframe correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    '''Make list of columns to be compared using t-tests'''
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    '''Call wrap_ttest, pass in formatted dataframe'''
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    '''Print results, if anything significant was found'''
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")

except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  KRAS
A1BG did not match any columns in phosphoproteomics dataframe. A1BG_phosphoproteomics column inserted, but filled with NaN.
A2M did not match any columns in phosphoproteomics dataframe. A2M_phosphoproteomics column inserted, but filled with NaN.
A2ML1 did not match any columns in phosphoproteomics dataframe. A2ML1_phosphoproteomics column inserted, but filled with NaN.
A4GALT did not match any columns in phosphoproteomics dataframe. A4GALT_phosphoproteomics column inserted, but filled with NaN.
AADAT did not match any columns in phosphoproteomics dataframe. AADAT_phosphoproteomics column inserted, but filled with NaN.
AAMP did not match any columns in phosphoproteomics dataframe. AAMP_phosphoproteomics column inserted, but filled with NaN.
AAR2 did not match any columns in phosphoproteomics dataframe. AAR2_phosphoproteomics column inserted, but filled with NaN.
AARS2 did not match any columns in phosphoproteomics dataframe. AARS2_phosphoproteomics column inserted, but fill

ACSL1 did not match any columns in phosphoproteomics dataframe. ACSL1_phosphoproteomics column inserted, but filled with NaN.
ACSM3 did not match any columns in phosphoproteomics dataframe. ACSM3_phosphoproteomics column inserted, but filled with NaN.
ACSS1 did not match any columns in phosphoproteomics dataframe. ACSS1_phosphoproteomics column inserted, but filled with NaN.
ACTA1 did not match any columns in phosphoproteomics dataframe. ACTA1_phosphoproteomics column inserted, but filled with NaN.
ACTA2 did not match any columns in phosphoproteomics dataframe. ACTA2_phosphoproteomics column inserted, but filled with NaN.
ACTB did not match any columns in phosphoproteomics dataframe. ACTB_phosphoproteomics column inserted, but filled with NaN.
ACTG1 did not match any columns in phosphoproteomics dataframe. ACTG1_phosphoproteomics column inserted, but filled with NaN.
ACTG2 did not match any columns in phosphoproteomics dataframe. ACTG2_phosphoproteomics column inserted, but filled with

AGPAT5 did not match any columns in phosphoproteomics dataframe. AGPAT5_phosphoproteomics column inserted, but filled with NaN.
AGR2 did not match any columns in phosphoproteomics dataframe. AGR2_phosphoproteomics column inserted, but filled with NaN.
AGR3 did not match any columns in phosphoproteomics dataframe. AGR3_phosphoproteomics column inserted, but filled with NaN.
AGT did not match any columns in phosphoproteomics dataframe. AGT_phosphoproteomics column inserted, but filled with NaN.
AHCY did not match any columns in phosphoproteomics dataframe. AHCY_phosphoproteomics column inserted, but filled with NaN.
AHSP did not match any columns in phosphoproteomics dataframe. AHSP_phosphoproteomics column inserted, but filled with NaN.
AIFM2 did not match any columns in phosphoproteomics dataframe. AIFM2_phosphoproteomics column inserted, but filled with NaN.
AIG1 did not match any columns in phosphoproteomics dataframe. AIG1_phosphoproteomics column inserted, but filled with NaN.
AK4 

ANGEL2 did not match any columns in phosphoproteomics dataframe. ANGEL2_phosphoproteomics column inserted, but filled with NaN.
ANGPT2 did not match any columns in phosphoproteomics dataframe. ANGPT2_phosphoproteomics column inserted, but filled with NaN.
ANGPT4 did not match any columns in phosphoproteomics dataframe. ANGPT4_phosphoproteomics column inserted, but filled with NaN.
ANGPTL1 did not match any columns in phosphoproteomics dataframe. ANGPTL1_phosphoproteomics column inserted, but filled with NaN.
ANGPTL4 did not match any columns in phosphoproteomics dataframe. ANGPTL4_phosphoproteomics column inserted, but filled with NaN.
ANKRD16 did not match any columns in phosphoproteomics dataframe. ANKRD16_phosphoproteomics column inserted, but filled with NaN.
ANKRD2 did not match any columns in phosphoproteomics dataframe. ANKRD2_phosphoproteomics column inserted, but filled with NaN.
ANKRD22 did not match any columns in phosphoproteomics dataframe. ANKRD22_phosphoproteomics column

ARFRP1 did not match any columns in phosphoproteomics dataframe. ARFRP1_phosphoproteomics column inserted, but filled with NaN.
ARG2 did not match any columns in phosphoproteomics dataframe. ARG2_phosphoproteomics column inserted, but filled with NaN.
ARIH2 did not match any columns in phosphoproteomics dataframe. ARIH2_phosphoproteomics column inserted, but filled with NaN.
ARL1 did not match any columns in phosphoproteomics dataframe. ARL1_phosphoproteomics column inserted, but filled with NaN.
ARL15 did not match any columns in phosphoproteomics dataframe. ARL15_phosphoproteomics column inserted, but filled with NaN.
ARL4A did not match any columns in phosphoproteomics dataframe. ARL4A_phosphoproteomics column inserted, but filled with NaN.
ARL4C did not match any columns in phosphoproteomics dataframe. ARL4C_phosphoproteomics column inserted, but filled with NaN.
ARL4D did not match any columns in phosphoproteomics dataframe. ARL4D_phosphoproteomics column inserted, but filled with

...

WFDC2 did not match any columns in phosphoproteomics dataframe. WFDC2_phosphoproteomics column inserted, but filled with NaN.
WIF1 did not match any columns in phosphoproteomics dataframe. WIF1_phosphoproteomics column inserted, but filled with NaN.
WNT11 did not match any columns in phosphoproteomics dataframe. WNT11_phosphoproteomics column inserted, but filled with NaN.
WNT2 did not match any columns in phosphoproteomics dataframe. WNT2_phosphoproteomics column inserted, but filled with NaN.
WNT2B did not match any columns in phosphoproteomics dataframe. WNT2B_phosphoproteomics column inserted, but filled with NaN.
WNT4 did not match any columns in phosphoproteomics dataframe. WNT4_phosphoproteomics column inserted, but filled with NaN.
WNT5A did not match any columns in phosphoproteomics dataframe. WNT5A_phosphoproteomics column inserted, but filled with NaN.
WNT5B did not match any columns in phosphoproteomics dataframe. WNT5B_phosphoproteomics column inserted, but filled with NaN

ZNF157 did not match any columns in phosphoproteomics dataframe. ZNF157_phosphoproteomics column inserted, but filled with NaN.
ZNF17 did not match any columns in phosphoproteomics dataframe. ZNF17_phosphoproteomics column inserted, but filled with NaN.
ZNF207 did not match any columns in phosphoproteomics dataframe. ZNF207_phosphoproteomics column inserted, but filled with NaN.
ZNF224 did not match any columns in phosphoproteomics dataframe. ZNF224_phosphoproteomics column inserted, but filled with NaN.
ZNF248 did not match any columns in phosphoproteomics dataframe. ZNF248_phosphoproteomics column inserted, but filled with NaN.
ZNF253 did not match any columns in phosphoproteomics dataframe. ZNF253_phosphoproteomics column inserted, but filled with NaN.
ZNF3 did not match any columns in phosphoproteomics dataframe. ZNF3_phosphoproteomics column inserted, but filled with NaN.
ZNF302 did not match any columns in phosphoproteomics dataframe. ZNF302_phosphoproteomics column inserted, but

ZNF888 did not match any columns in phosphoproteomics dataframe. ZNF888_phosphoproteomics column inserted, but filled with NaN.
ZNRD1 did not match any columns in phosphoproteomics dataframe. ZNRD1_phosphoproteomics column inserted, but filled with NaN.
ZSCAN12 did not match any columns in phosphoproteomics dataframe. ZSCAN12_phosphoproteomics column inserted, but filled with NaN.
ZSCAN26 did not match any columns in phosphoproteomics dataframe. ZSCAN26_phosphoproteomics column inserted, but filled with NaN.
ZSWIM9 did not match any columns in phosphoproteomics dataframe. ZSWIM9_phosphoproteomics column inserted, but filled with NaN.
ZWILCH did not match any columns in phosphoproteomics dataframe. ZWILCH_phosphoproteomics column inserted, but filled with NaN.
ZWINT did not match any columns in phosphoproteomics dataframe. ZWINT_phosphoproteomics column inserted, but filled with NaN.
ZYG11B did not match any columns in phosphoproteomics dataframe. ZYG11B_phosphoproteomics column inserte

/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3157: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


                          Comparison       P_Value
0      DTNBP1-S297_phosphoproteomics  1.239489e-09
1      IGF2R-S2479_phosphoproteomics  3.548371e-09
2      DTNBP1-S300_phosphoproteomics  6.909004e-09
3        TPR-S1662_phosphoproteomics  1.383305e-08
4     PLEKHS1-S185_phosphoproteomics  2.395489e-08
5     LRRFIP1-S300_phosphoproteomics  3.795856e-08
6         KRT8-Y38_phosphoproteomics  1.362150e-07
7         KRT8-S63_phosphoproteomics  3.383255e-07
8       PLEC-S4408_phosphoproteomics  3.449881e-07
9      MACF1-S1377_phosphoproteomics  3.491278e-07
10       HDGF-S130_phosphoproteomics  3.878632e-07
11     PRKAA1-S521_phosphoproteomics  4.422808e-07
12  ARHGEF18-S1160_phosphoproteomics  5.761535e-07
13   IVNS1ABP-S269_phosphoproteomics  6.137179e-07
14     TNRC6B-S803_phosphoproteomics  6.748903e-07





## Colon

In [23]:
try:
    print("\nGene: ", gene)

    '''Use all proteins'''
    proteomics = co.get_proteomics()
    all_proteins = proteomics.columns
    #all_proteins = all_proteins[:100]

    '''Create dataframe in order to do comparisons with wrap_ttest'''
    protdf = co.append_mutations_to_omics(mutation_genes=[gene], omics_df_name=omics, omics_genes=all_proteins)
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    '''Create the binary valued column needed to do the comparison'''
    for ind, row in protdf.iterrows():
        if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
            protdf.at[ind,'Label'] = 'Mutated'
        else:
            protdf.at[ind,'Label'] = 'Wildtype'

    '''Format the datafram correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    '''Make list of columns to be compared using t-tests'''
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    '''Call wrap_ttest, pass in formatted dataframe'''
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    '''Print results, if anything significant was found'''
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")

except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  KRAS
A1BG did not match any columns in phosphoproteomics dataframe. A1BG_phosphoproteomics column inserted, but filled with NaN.
A1CF did not match any columns in phosphoproteomics dataframe. A1CF_phosphoproteomics column inserted, but filled with NaN.
A2M did not match any columns in phosphoproteomics dataframe. A2M_phosphoproteomics column inserted, but filled with NaN.
AACS did not match any columns in phosphoproteomics dataframe. AACS_phosphoproteomics column inserted, but filled with NaN.
AAMP did not match any columns in phosphoproteomics dataframe. AAMP_phosphoproteomics column inserted, but filled with NaN.
AAR2 did not match any columns in phosphoproteomics dataframe. AAR2_phosphoproteomics column inserted, but filled with NaN.
AARS2 did not match any columns in phosphoproteomics dataframe. AARS2_phosphoproteomics column inserted, but filled with NaN.
AASDHPPT did not match any columns in phosphoproteomics dataframe. AASDHPPT_phosphoproteomics column inserted, but fill

ACTR1B did not match any columns in phosphoproteomics dataframe. ACTR1B_phosphoproteomics column inserted, but filled with NaN.
ACTR3 did not match any columns in phosphoproteomics dataframe. ACTR3_phosphoproteomics column inserted, but filled with NaN.
ACTR3B did not match any columns in phosphoproteomics dataframe. ACTR3B_phosphoproteomics column inserted, but filled with NaN.
ACTR5 did not match any columns in phosphoproteomics dataframe. ACTR5_phosphoproteomics column inserted, but filled with NaN.
ACTR6 did not match any columns in phosphoproteomics dataframe. ACTR6_phosphoproteomics column inserted, but filled with NaN.
ACY1 did not match any columns in phosphoproteomics dataframe. ACY1_phosphoproteomics column inserted, but filled with NaN.
ACYP1 did not match any columns in phosphoproteomics dataframe. ACYP1_phosphoproteomics column inserted, but filled with NaN.
ACYP2 did not match any columns in phosphoproteomics dataframe. ACYP2_phosphoproteomics column inserted, but filled 

AKR1D1 did not match any columns in phosphoproteomics dataframe. AKR1D1_phosphoproteomics column inserted, but filled with NaN.
AKR7A3 did not match any columns in phosphoproteomics dataframe. AKR7A3_phosphoproteomics column inserted, but filled with NaN.
ALAS1 did not match any columns in phosphoproteomics dataframe. ALAS1_phosphoproteomics column inserted, but filled with NaN.
ALCAM did not match any columns in phosphoproteomics dataframe. ALCAM_phosphoproteomics column inserted, but filled with NaN.
ALDH1A2 did not match any columns in phosphoproteomics dataframe. ALDH1A2_phosphoproteomics column inserted, but filled with NaN.
ALDH1A3 did not match any columns in phosphoproteomics dataframe. ALDH1A3_phosphoproteomics column inserted, but filled with NaN.
ALDH1L1 did not match any columns in phosphoproteomics dataframe. ALDH1L1_phosphoproteomics column inserted, but filled with NaN.
ALDH1L2 did not match any columns in phosphoproteomics dataframe. ALDH1L2_phosphoproteomics column ins

AP3M2 did not match any columns in phosphoproteomics dataframe. AP3M2_phosphoproteomics column inserted, but filled with NaN.
AP3S1 did not match any columns in phosphoproteomics dataframe. AP3S1_phosphoproteomics column inserted, but filled with NaN.
AP4M1 did not match any columns in phosphoproteomics dataframe. AP4M1_phosphoproteomics column inserted, but filled with NaN.
AP4S1 did not match any columns in phosphoproteomics dataframe. AP4S1_phosphoproteomics column inserted, but filled with NaN.
AP5B1 did not match any columns in phosphoproteomics dataframe. AP5B1_phosphoproteomics column inserted, but filled with NaN.
AP5M1 did not match any columns in phosphoproteomics dataframe. AP5M1_phosphoproteomics column inserted, but filled with NaN.
AP5S1 did not match any columns in phosphoproteomics dataframe. AP5S1_phosphoproteomics column inserted, but filled with NaN.
AP5Z1 did not match any columns in phosphoproteomics dataframe. AP5Z1_phosphoproteomics column inserted, but filled wi

ARSA did not match any columns in phosphoproteomics dataframe. ARSA_phosphoproteomics column inserted, but filled with NaN.
ARSB did not match any columns in phosphoproteomics dataframe. ARSB_phosphoproteomics column inserted, but filled with NaN.
ARSD did not match any columns in phosphoproteomics dataframe. ARSD_phosphoproteomics column inserted, but filled with NaN.
ARSE did not match any columns in phosphoproteomics dataframe. ARSE_phosphoproteomics column inserted, but filled with NaN.
ASAH1 did not match any columns in phosphoproteomics dataframe. ASAH1_phosphoproteomics column inserted, but filled with NaN.
ASB9 did not match any columns in phosphoproteomics dataframe. ASB9_phosphoproteomics column inserted, but filled with NaN.
ASCC1 did not match any columns in phosphoproteomics dataframe. ASCC1_phosphoproteomics column inserted, but filled with NaN.
ASL did not match any columns in phosphoproteomics dataframe. ASL_phosphoproteomics column inserted, but filled with NaN.
ASNA1 

...

URM1 did not match any columns in phosphoproteomics dataframe. URM1_phosphoproteomics column inserted, but filled with NaN.
UROD did not match any columns in phosphoproteomics dataframe. UROD_phosphoproteomics column inserted, but filled with NaN.
UROS did not match any columns in phosphoproteomics dataframe. UROS_phosphoproteomics column inserted, but filled with NaN.
USE1 did not match any columns in phosphoproteomics dataframe. USE1_phosphoproteomics column inserted, but filled with NaN.
USF1 did not match any columns in phosphoproteomics dataframe. USF1_phosphoproteomics column inserted, but filled with NaN.
USMG5 did not match any columns in phosphoproteomics dataframe. USMG5_phosphoproteomics column inserted, but filled with NaN.
USP12 did not match any columns in phosphoproteomics dataframe. USP12_phosphoproteomics column inserted, but filled with NaN.
USP27X did not match any columns in phosphoproteomics dataframe. USP27X_phosphoproteomics column inserted, but filled with NaN.


WDR45 did not match any columns in phosphoproteomics dataframe. WDR45_phosphoproteomics column inserted, but filled with NaN.
WDR45B did not match any columns in phosphoproteomics dataframe. WDR45B_phosphoproteomics column inserted, but filled with NaN.
WDR5 did not match any columns in phosphoproteomics dataframe. WDR5_phosphoproteomics column inserted, but filled with NaN.
WDR6 did not match any columns in phosphoproteomics dataframe. WDR6_phosphoproteomics column inserted, but filled with NaN.
WDR61 did not match any columns in phosphoproteomics dataframe. WDR61_phosphoproteomics column inserted, but filled with NaN.
WDR73 did not match any columns in phosphoproteomics dataframe. WDR73_phosphoproteomics column inserted, but filled with NaN.
WDR82 did not match any columns in phosphoproteomics dataframe. WDR82_phosphoproteomics column inserted, but filled with NaN.
WDR83 did not match any columns in phosphoproteomics dataframe. WDR83_phosphoproteomics column inserted, but filled with

ZFYVE27 did not match any columns in phosphoproteomics dataframe. ZFYVE27_phosphoproteomics column inserted, but filled with NaN.
ZG16 did not match any columns in phosphoproteomics dataframe. ZG16_phosphoproteomics column inserted, but filled with NaN.
ZIC4 did not match any columns in phosphoproteomics dataframe. ZIC4_phosphoproteomics column inserted, but filled with NaN.
ZMAT2 did not match any columns in phosphoproteomics dataframe. ZMAT2_phosphoproteomics column inserted, but filled with NaN.
ZMIZ1 did not match any columns in phosphoproteomics dataframe. ZMIZ1_phosphoproteomics column inserted, but filled with NaN.
ZMPSTE24 did not match any columns in phosphoproteomics dataframe. ZMPSTE24_phosphoproteomics column inserted, but filled with NaN.
ZNF135 did not match any columns in phosphoproteomics dataframe. ZNF135_phosphoproteomics column inserted, but filled with NaN.
ZNF143 did not match any columns in phosphoproteomics dataframe. ZNF143_phosphoproteomics column inserted, but

/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3157: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


No significant comparisons.


Replicate error with "-" in endometrial phosphoproteomics

In [24]:
import cptac
en = cptac.Endometrial()

proteomics = en.get_proteomics()
all_proteins = proteomics.columns
replicate_error_proteins = all_proteins[400:500]

prot_df = en.append_mutations_to_omics(mutation_genes=["KRAS"], omics_df_name="phosphoproteomics", omics_genes=replicate_error_proteins)


Checking that index is up-to-date...Loading acetylproteomics data...Loading clinical data...Loading CNA data...Loading definitions data...Loading miRNA data...Loading phosphoproteomics_gene data...Loading phosphoproteomics_site data...Loading proteomics data...Loading somatic_binary data...Loading somatic data...Loading transcriptomics_circular data...Loading transcriptomics_linear data...Formatting dataframes...ANG did not match any columns in phosphoproteomics dataframe. ANG_phosphoproteomics column inserted, but filled with NaN.
ANGEL2 did not match any columns in phosphoproteomics dataframe. ANGEL2_phosphoproteomics column inserted, but filled with NaN.
ANGPT2 did not match any columns in phosphoproteomics dataframe. ANGPT2_phosphoproteomics column inserted, but filled with NaN.
ANGPT4 did not match any columns in phosphoproteomics dataframe. ANGPT4_phosphoproteomics column inserted, but filled with NaN.
ANGPTL1 did not match any columns in phosphoproteomics dataframe. ANGPTL1_phos

ValueError: columns overlap but no suffix specified: Index(['ANKHD1-EIF4EBP3-S2539_phosphoproteomics',
       'ANKHD1-EIF4EBP3-S2556_phosphoproteomics',
       'ANKHD1-EIF4EBP3-S2598_phosphoproteomics',
       'ANKHD1-EIF4EBP3-T2543_phosphoproteomics'],
      dtype='object')